In [1]:
import pandas as pd
import numpy as np
import os
from osgeo import gdal
import scipy
import random
from numpy import random

In [2]:
CSV_Train_File = "E:\\TMP\\GIT-ML\\PlanetAmazon\\train_csv_v2\\train_v2.csv"

df = pd.read_csv(CSV_Train_File)
df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [3]:
all_tags = [item for sublist in list(df['tags'].apply(lambda row: row.split(" ")).values) for item in sublist]
all_tags = np.asarray(all_tags)
unique_tags = np.unique(all_tags)

In [4]:
for label in unique_tags:
    print("Learn", label)
    df[label] = pd.Series(0, index=df.index)
    df.loc[df['tags'].str.contains(label), label] = 1

Learn agriculture
Learn artisinal_mine
Learn bare_ground
Learn blooming
Learn blow_down
Learn clear
Learn cloudy
Learn conventional_mine
Learn cultivation
Learn habitation
Learn haze
Learn partly_cloudy
Learn primary
Learn road
Learn selective_logging
Learn slash_burn
Learn water


In [5]:
import numpy as np
import math

# labels_dict : {ind_label: count_label}
# mu : parameter to tune 

def create_class_weight(labels_dict,mu=0.15):
    total = sum(labels_dict.values())
    keys = labels_dict.keys()
    class_weight = dict()
    mutotal = 0.15*float(total)
    for key in keys:
        
        score = math.log(mutotal/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0

    return class_weight

# random labels_dict
labels_dict = {0: 12338, 1: 339, 2: 859, 3: 332, 4: 101, 5: 28203, 6: 2330, 7: 100, 8: 4547, 9: 3662, 10: 2695, 11:7251, 12: 37840, 13: 8076, 14:340, 15:209, 16:7262}
weight_pclass = create_class_weight(labels_dict)

In [6]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.models import load_model

Using TensorFlow backend.


In [8]:
def cnn_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17))
    model.add(Activation('sigmoid'))
    #model.add(Activation('softmax'))

    return model

In [13]:
def imgProcessingRGB(imgName, pathImg):
    
    img = gdal.Open(pathImg + imgName + '.tif')
    #img = gdal.Open(pathImg + imgName + '.jpg')
    imgData = img.ReadAsArray()

    r = imgData[2,:,:]
    g = imgData[1,:,:]
    b = imgData[0,:,:]
    #nir = imgData[3,:,:]
    
    #r = r/255.
    #g = g/255.
    #b = b/255.

    r = (r / 65535.)**(1/2.2)
    g = (g / 65535.)**(1/2.2)
    b = (b / 65535.)**(1/2.2)

    newLayers = np.dstack([r,g,b])
    return newLayers

In [14]:
def generatorRGB(features, labels, batch_size, pathImg):
    batch_features = np.zeros((batch_size, 256, 256, 3))
    batch_labels = np.zeros((batch_size,17))
    while True:
        for i in range(batch_size):
            # choose random index in features
            index= random.choice(len(features),1)[0]
            batch_features[i] = imgProcessingRGB(features[index], pathImg)
            batch_labels[i] = labels[index]
        yield (batch_features, batch_labels)

In [15]:
saveModelPath = "E:\\TMP\GIT-ML\\PlanetAmazon\\keras_models\\"
labels = df.values[:,2:]
features = df['image_name'].values
print(labels.shape)
print(features.shape)
input_shape = (256, 256, 3)
#train_path = 'E:\\TMP\\GIT-ML\\PlanetAmazon\\train-jpg-v2\\'
train_path = 'C:\\Temp\\train-tif-v2\\'
test_path = 'E:\\TMP\GIT-ML\\PlanetAmazon\\test-jpg-v2\\'

(40479, 17)
(40479,)


In [ ]:
model = cnn_model()
model.compile(loss='binary_crossentropy',
                #optimizer='rmsprop',
                optimizer='adam',
                metrics=['accuracy'])

batch_size = 16
pathImg = train_path
model.fit_generator( generatorRGB(features,labels,batch_size, pathImg), 
                    steps_per_epoch = 5400, epochs = 17, class_weight = weight_pclass,
                    verbose=1, callbacks=None, validation_data=None)
model.save(saveModelPath +  "MultyLabelsRGBvW2.h5")

Epoch 1/17
3999/5400 [=====================>........] - ETA: 289s - loss: 0.3852 - acc: 0.8935